In [1]:

import numpy as np

In [2]:
import gym

env = gym.make("CarRacing-v0")

In [3]:
#env.reset()
#for i in range(1000):
    #env.render()
#    action = env.action_space.sample()
#    env.step(action)

In [4]:
class ReplayMemory:
    #rotating buffer of size N
    def __init__(self,N,batch_size=1000):
        self.memory = []
        self.size = N
        self.batch_size =batch_size
    
    def add(self,experience):
        self.memory.append(experience)
        if(len(self.memory) > self.size):
            del self.memory[0]
    
    def replay(self):    
        #if(len(self.memory) < self.batch_size):
        #    return []
        #relay everything stored in small minibatches 
        # (currently of lenght 1)
        # in a random order to reduce correlation
        output = np.array(self.memory)
        np.random.shuffle(output)
        return output#[:self.batch_size]

#Define what our experience looks like
#[state,action,reward,next_state]

In [5]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        
        # assuming our input is 16
        # and our output is 4
        self.dense1 = nn.Linear(16,256)
        self.dense2 = nn.Linear(256,4)
        
    
    
    def forward(self,x):
        if(type(x) != torch.int32):
            x = int(x)
            x = to_onehot(16,x)
            
        out = F.sigmoid(self.dense1(x))
        out = self.dense2(out)
        return out

NameError: name 'nn' is not defined

In [ ]:
future_discount = .99

def learn_from_data(model,target,data,optimizer):
    for experience in data:
        #compute the loss from
        state,reward,action,next_state = experience
        state = int(state)
        next_state = int(next_state)
        action=int(action)
        
        
        #belman equation
        
        chosen_action_based_on_reward = model.forward(state)[action]
        expected_reward = reward + target.forward(next_state).max(0)[0].detach_()
        
        loss = F.l1_loss(chosen_action_based_on_reward.double(),expected_reward.double())
        #we want the chosen_action based on reward to match 
        #the reward of being in the next state and the reward given
        
        #print(reward)
        
        # must zero gradients before backprop
        # for pytorch 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def get_best_action(model,state):
    #get our best action from our learner
    #print("state given:",state)
    action = model.forward(state).max(0)[1]
    action = int(action)
    #print("State:",state,"Action:",action)
    return action
    
def update_target(model,target):
    target.load_state_dict(model.state_dict())
    #print("model:",model.state_dict())
    #print("target:",target.state_dict())
    return target
    
    

def show_progress(episode,rewards,fidelity):
    clear_output()
    print("Episode:",episode)
    average =  [sum(rewards[fidelity*k:fidelity*(k+1)])/fidelity for k in range(int(len(rewards)/fidelity))]
    print("Best", max(average))
    plt.plot(average)
    plt.show()

In [ ]:
num_episodes = 250000
max_steps_per_episode = 100

learning_rate = 0.05

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.0001

memory = ReplayMemory(10000)

model= DQN()
rewards = []
optimizer = optim.SGD(model.parameters(),lr=.01)
target = DQN()

        

for episode in range(num_episodes):
    state = env.reset()
    #print("Episode: %d"%episode)
    
    done = False
    
    total_reward = 0
    
    game = []
    
    for step in range(max_steps_per_episode):
        #env.render()
        #clear_output()
        
        uniform_random_variable= random.uniform(0,1)
        
        if(uniform_random_variable > exploration_rate):
            
            action = get_best_action(model,state)
            
        else:
            #other wise explore randomly
            action = env.action_space.sample()
            
            
        #now that the action is chosen we take it
        new_state, reward, done, info = env.step(action)
        
        total_reward += reward
        
        #we save this experience to learn from later
        experience=[state,total_reward,action,new_state]
        
        #game.append(experience)
        memory.add(experience)
        state = new_state
        
        if(done):
            break
    
    #if(total_reward > 0):
        #[memory.add(experience) for experience in game]
        #print(len(memory.memory))
    
    #Decay the exploration rate after each loop
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    rewards.append(total_reward)
    
    if(episode % 100 == 0):
        learn_from_data(model,model,memory.replay(),optimizer)
        
        if(episode % 500==0 and episode!=0):
            #update the target model to match the model
            #target = update_target(model,target)
            show_progress(episode,rewards,500)
            print("Exploration_rate:",exploration_rate)
            
            #if(episode%1000==0):
        